# ConversationEntityMemory
대화에서 특정 엔티티에 대한 주어진 사실을 기억.

엔티티에 대한 정보를 추출하고(LLM 사용) 시간이 지남에 따라 해당 엔티티에 대한 지식을 축적(역시 LLM사용)

- 즉,글자수 등으로 기억하는게 아니라 대화에서 Entity(주요 핵심 정보)를 추출하여 추후에 필요한 정보인것으로 생각하고 압축해서 가지고 있는것이다.
- 대화가 길어지더라도 효율적일 수 있다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

Entity 메모리를 효과적으로 사용하기 위해 제공하는 프롬프트를 사용.

In [5]:
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

In [7]:
llm = ChatOpenAI(model="gpt-4o-mini")
# ConversationChain을 생성한다.
# llm이 모두 같을 필요는 없다, 대화내용을 추출할때는 저비용의 llm을 사용해도 된다.
conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

`ConversationEntityMemory`는 주요 Entity를 별도로 저장한다

In [9]:
conversation.predict(
    input="테디와 재호는 한 회사에서 일하는 동료입니다."
    "테디는 개발자이고 재호는 디자이너입니다. "
    "그들은최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다."
)

'그렇군요! 테디와 재호가 함께 회사를 차릴 계획을 세우고 있다니 흥미로운 소식이네요. 두 사람의 전문성이 잘 결합된다면 멋진 결과를 만들어 낼 수 있을 것 같습니다. 어떤 분야의 사업을 시작할 계획인지, 아니면 어떤 아이디어를 구상하고 있는지 궁금하네요!'

In [10]:
# entity memory 를 출력합니다.
conversation.memory.entity_store.store

{'테디': '테디는 개발자로서 재호와 함께 일하며, 최근 회사에서 퇴사하고 자신들의 회사를 차릴 계획을 세우고 있습니다.',
 '재호': '재호는 테디와 함께 한 회사에서 일하는 디자이너이며, 최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다.'}